In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

df = pd.read_csv('/content/movies_metadata.csv')
df.loc[df['release_date'] == 'TV Movie 2019', 'release_date'] = '2019'
df['release_date'] = pd.to_datetime(df['release_date'], infer_datetime_format=True, errors='coerce')
df.drop(columns='adult', inplace=True)
df['year'] = df['release_date'].dt.year
df['year'] = df['year'].astype(float)
df_fresh = pd.read_csv('/content/IMDb movies.csv', low_memory=False)
df_fresh.loc[df_fresh['year'] == 'TV Movie 2019', 'year'] = 2019
df_fresh['year'] = df_fresh['year'].astype(float)
a = df.loc[df['revenue'].isnull()].index
df.drop(labels=a, inplace=True)
df.reset_index(inplace=True)
df.drop_duplicates(inplace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
alpha = df.loc[df['revenue'] == 0, 'title']

for x in alpha:
  b = df_fresh.loc[df_fresh['title'] == x, 'worlwide_gross_income']
  c = df_fresh.loc[df_fresh['title'] == x, 'year']
  for y, z in enumerate(b):
    df.loc[(df['original_title']==x) & (df['year'] ==c.iloc[y]), 'revenue'] = b.iloc[y]

In [4]:
alpha = df.loc[df['revenue'] == 0].index
df.drop(labels=alpha, inplace=True)

In [5]:
df_act_dir = pd.read_csv('/content/IMDb movies.csv', low_memory=False)

df_act_dir.drop(columns = ['imdb_title_id', 'date_published', 'genre', 'country', 'language', 
                           'description', 'avg_vote', 'votes', 'budget', 'usa_gross_income', 'worlwide_gross_income', 
                           'metascore', 'reviews_from_users', 'reviews_from_critics', 'original_title'], inplace=True)
df = pd.merge(df, df_act_dir, how='inner', on='title')

In [6]:
features_to_drop = ['belongs_to_collection', 'homepage', 'imdb_id', 'overview', 
                    'tagline', 'title', 'poster_path', 'original_title', 'production_companies', 'status']
df_x = df.drop(columns = features_to_drop)

In [7]:
ak = []
for x in df_x['production_countries']:
    x = x.split(':')
    if len(x)<=1:
        ak.append('unknown')
    else:
        x = x[-1].strip('}]')
        x = x.strip()
        ak.append(x)

df_x['production_countries'] = ak

In [8]:
ab = []
ad = []
for x in range(len(df['genres'])):
    ac = []
    ae = []
    y = df['genres'][x]
    g = y.split(':')
    for z in range(len(g)):
        if z%2 == 1:
            h = g[z].split()
            h = h[0].strip(',')
            ac.append(h)
    ab.append(ac)

            
df_x['genres'] = ab   

In [9]:
ab = []
for x in df_x['genres']:
    if len(x) == 0:
        ab.append(0)
    else:
        for y in range(len(x)):
            if y ==0:
                g = x[y]
            else:
                g = g + x[y]
        ab.append(g)

df_x['genres'] = ab      

In [10]:
def encoder(n, d):
    
    h = n.copy()
    aliup1 = h[d].value_counts()
    mal = []
    ifi = []
    for z in aliup1.index:
        if aliup1.loc[z] < 30:
          h.loc[h[d] == z, d] = 'other'
    aliup = h[d].value_counts().index
    for z in range(len(aliup)):
        mal.append(z + 1)
    aliup = pd.DataFrame(aliup)
    aliup['enc'] = mal
    for x in aliup[0]:
        b = aliup.loc[aliup[0]==x, 'enc']
        b = b.iloc[0]
        h.loc[h[d] == x, d] = b
    return (h[d])

In [11]:
a = df_x.loc[df['revenue'].isnull()].index
df_x.drop(labels=a, inplace=True)
df_x.drop(columns=['year_x', 'year_y'], inplace=True)

In [12]:
ab = []
df_x['revenue'] = df_x['revenue'].astype(str)
for x in df_x['revenue']:
  x = x.strip('$')
  x = x.strip('INR')
  ab.append(x)

df_x['revenue'] = ab
df_x['revenue'] = df_x['revenue'].astype(float)

df_x['release_date'] = pd.to_datetime(df_x['release_date'], infer_datetime_format=True)
df_x['year'] = df_x['release_date'].dt.year
df_x['month'] = df_x['release_date'].dt.month
df_x['day'] = df_x['release_date'].dt.day
df_x.drop(columns='release_date', inplace=True)

In [13]:
df_x['original_language'] = encoder(df_x, 'original_language')
df_x['video'] = encoder(df_x, 'video')
df_x['genres'] = encoder(df_x, 'genres')
df_x['production_countries'] = encoder(df_x, 'production_countries')
df_x['spoken_languages'] = encoder(df_x, 'spoken_languages')
#df_x['director'] = encoder(df_x, 'director')
df_x['production_company'] = encoder(df_x, 'production_company')
#df_x['writer'] = encoder(df_x, 'writer')
#df_x['actors'] = encoder(df_x, 'actors')

In [14]:
top_1000 = pd.read_csv('/content/Top 1000 Actors and Actresses.csv', encoding='latin-1')

In [15]:
abb = []
for x in top_1000['Name']:
  abb.append(x)

In [16]:
df_x.loc[df_x['actors'].isnull(), 'actors'] = 'none'

ab = []

for x in df_x['actors']:
  ac = []
  x = x.split(',')
  for y in x:
    y = y.strip()
    if y in abb:
      ac.append(1)
  b = np.sum(ac)
  ab.append(b)

In [17]:
df_x['actors'] = ab

In [18]:
top_dir = pd.read_csv('/content/1000 the Best Directors.csv', encoding='latin-1')

In [19]:
top_dir.columns
abb = []
for x in top_dir['Name']:
  abb.append(x)

In [20]:
df_x['director'] = df_x['director'].astype(str)
ab = []
for x in df_x['director']:
  ac = []
  x = x.split(',')
  for y in x:
    y = y.strip()
    if y in abb:
      ac.append(1)
  b = np.sum(ac)
  ab.append(b)

In [21]:
df_x['director'] = ab

In [22]:
df_x = df_x.drop(columns='writer')

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

In [ ]:
df_x.drop(columns=['vote_average', 'vote_count'], inplace=True)

In [118]:
X = df_x.drop(columns='revenue')
y = df_x['revenue']

In [119]:
xtrain, X_test, ytrain, y_test = train_test_split(X, y, test_size = .2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(xtrain, ytrain, test_size = .2, random_state=42)

In [120]:
baseline = np.mean(y_train)
baseline_train = [baseline for x in range(len(y_train))]
baseline_train = np.array(baseline_train)
baseline_acc = mean_absolute_error(y_train, baseline_train)

In [121]:
baseline_acc

52169022.79024799

In [122]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

pipeline=make_pipeline(SimpleImputer(), RandomForestRegressor(n_estimators = 150, max_depth=20, random_state=41))
pipeline.fit(X_train, y_train)

pipeline.score(X_train, y_train)

0.9650427970579432

In [123]:
pipeline.score(X_val, y_val)

0.7683953071882952

In [124]:
y_pred = pipeline.predict(X_val)
display(mean_absolute_error(y_val, y_pred))
display(mean_squared_error(y_val, y_pred))

18460245.52430295

3057441624891550.0

In [125]:
for x in X_train.columns:
  X_train[x] = X_train[x].astype(float)
  X_val[x] = X_val[x].astype(float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [126]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, y_train)

In [127]:
params = {'objective': 'reg:squarederror'}
bst = xgb.train(params=params, dtrain=dtrain, num_boost_round=150)
dtest = xgb.DMatrix(X_val, label=y_val)

In [128]:
alpha = bst.predict(dtest)

In [129]:
r2_score(y_val, alpha)

0.7600850826117129

In [130]:
mean_absolute_error(y_val, alpha)

18545075.687065266

In [131]:
mean_squared_error(y_val, alpha)

3167145906891121.0

In [ ]:
from xgboost import XGBRegressor

model = XGBRegressor(
    n_estimators=1000, 
    max_depth=7,       
    learning_rate=0.3, 
    n_jobs=-1
)

eval_set = [(X_train, y_train),
            (X_val, y_val)]

model.fit(X_train, y_train,
          eval_set=eval_set,
          eval_metric='rmse',
          early_stopping_rounds=70)

In [133]:
y_pred = model.predict(X_val)
r2_score(y_val, y_pred)

0.7448101971823162

In [134]:
display(mean_absolute_error(y_val, y_pred))
display(mean_squared_error(y_val, y_pred))


19237787.338634748

3368791521063784.0